Research Question: Can student-written RateMyProfessors comments be used to predict numerical professor ratings?

The dataset used in this project will be a publicly available data set from Huggingface, you can access the dataset from this link: https://huggingface.co/datasets/ZephyrUtopia/ratemyprofessors_reviews
If you see GPT anywhere please understand that I took some help from GPT to write that piece of code

Load the dataset from Hugging Face

In [ ]:
from datasets import load_dataset
#Load the dataset from the site of Hugging Face
dataset = load_dataset("ZephyrUtopia/ratemyprofessors_reviews")
print(dataset)

Convert the dataset to pandas array

In [ ]:
import pandas as pd
#GPT: Make a dataframe df which has all the data from the dataset of Hugging face
#as pandas array
df = pd.concat(
    [dataset[split].to_pandas() for split in dataset.keys()],
    ignore_index=True
)
print(df.columns)
print(df.shape)

Some basic printing functions to see some of the observations

In [ ]:
#Don't get confused with the output, in some of the cases when pandas cannot squeeze data, it divides
#the data in the way as shows in the output
print(df)
print(df["text"].head())
print(df["rating"].value_counts())
print(df["rating"].describe()) #rating is still not numeric, so this does not give too much information
print(df.describe())
print(pd.to_numeric(df["rating"], errors="coerce").describe()) #converting rating to numerical value coerce will assign 
#NaN value to missing or invalid data, also here describe will give all the parameters it gives for numericals

Add some demo of the plots using seaborn

In [ ]:
%pip install seaborn
%pip install matplotlib
import seaborn as sns
import matplotlib.pyplot as plt

Bar chart for frequency count of each of the ratings

In [ ]:
#GPT: Create a bar graph showing the frequency of each rating value
#Ratings are ordered in ascending order
ax = sns.countplot(
    data=df,
    x="rating",
    order=sorted(df["rating"].dropna().unique())
)
#Add frequency labels on top of each bar
#In matplot/seaborn, bars are store in "containers", so this loop goes to every bar in iterations
for container in ax.containers:
    ax.bar_label(container)
#Display the plot, in jupyter notebook this graph is shown even without the following line, but it might not be the case for .py or
#other coding environments, so writing the following line ensures portability
plt.show()
